# Voltage Regulator Waveform Demo

This notebook demonstrates publishing and reading back parametric waveform data with the NI Measurement Data Store. To demonstrate this, it uses simulated data of the dynamic response of a voltage regulator circuit. The simulated voltage regulator is set to regulate to 5V. The input voltage is the measurement parameter and varies from 4.5V to 12V. The data shows how the regulator can only regulate voltages within a certain range. If the input voltage is too large, it will not be able to pin the output voltage to its intended 5V.

# Set up the test station and other metadata (one time setup)

Setting up the metadata for operators, test stations, hardware, etc will typically be done once. We establish 'aliases' which are human-readable strings that can be used to refer to metadata later.

In [ ]:
from ni.datastore import (
    Client,
    HardwareItem,
    Operator,
    SoftwareItem,
    TestStation,
    Uut,
    UutInstance,
)

# Constants
ALIAS_JAMES_BOWERY = "Operator_BoweryJ"
ALIAS_TEST_STATION_12 = "TestStation_12"
ALIAS_UUT_NI_6508 = "UUT_NI-6508"
ALIAS_SOFTWARE_ITEM_WINDOWS_11 = "Windows_11_23H2"
ALIAS_SOFTWARE_ITEM_PYTHON_3_12 = "Python_3.12"
ALIAS_HARDWARE_ITEM_PXIe_5171 = "HardwareItem_PXIe-5171"

client = Client()

# Create alias for Operator "James Bowery"
operator = Operator(operator_name="James Bowery", role="Test Architect")
client.create_operator(operator)
client.create_alias(ALIAS_JAMES_BOWERY, operator)

# Create alias for Test Station "TestStation_12"
test_station = TestStation(test_station_name="TestStation_12")
client.create_test_station(test_station)
client.create_alias(ALIAS_TEST_STATION_12, test_station)

# Create alias for UUT instance "A861-42367"
uut = Uut(model_name="NI-6508", family="Digital")
uut_id = client.create_uut(uut)
client.create_alias(ALIAS_UUT_NI_6508, uut)

# Create aliases for SoftwareItems
software_item = SoftwareItem(product="Windows 11 Enterprise", version="23H2")
client.create_software_item(software_item)
client.create_alias(ALIAS_SOFTWARE_ITEM_WINDOWS_11, software_item)
software_item_2 = SoftwareItem(product="Python", version="3.12")
client.create_software_item(software_item_2)
client.create_alias(ALIAS_SOFTWARE_ITEM_PYTHON_3_12, software_item_2)

# Create aliases for HardwareItem
scope = HardwareItem(
    manufacturer="NI",
    model="PXIe-5171",
    serial_number="1933B4E",
)
client.create_hardware_item(scope)
client.create_alias(ALIAS_HARDWARE_ITEM_PXIe_5171, scope)



## Set up the test result (each test run)

This setup would be done on each test run. Each time we're going to run a test, we create a `TestResult` using the aliases created for the system from the step above.

In [ ]:
from ni.datastore import TestResult

# Create an instance of the 'NI-6508' on each run
uut_instance = UutInstance(uut_id=ALIAS_UUT_NI_6508, serial_number="A861-42367")
uut_instance_id = client.create_uut_instance(uut_instance)

test_result = TestResult(
    uut_instance_id=uut_instance_id,
    operator_id=ALIAS_JAMES_BOWERY,
    test_station_id=ALIAS_TEST_STATION_12,
    software_item_ids=[
        ALIAS_SOFTWARE_ITEM_WINDOWS_11,
        ALIAS_SOFTWARE_ITEM_PYTHON_3_12,
    ],
    hardware_item_ids=[ALIAS_HARDWARE_ITEM_PXIe_5171],
    test_result_name="voltage regulator waveforms",
)
test_result_id = client.create_test_result(test_result)


## Simulate Data and Publish with Conditions

This step simulates the data response for the voltage regulator and publishes the `AnalogWaveforms` with conditions (parameter values) for the input voltage. There would likely be data published from multiple test runs with the same `TestResult / test_result_id`.

In [ ]:
from datetime import timezone
import hightime as ht
import numpy as np
import plotly.graph_objects as go

from ni.datastore import Step
from nitypes.scalar import Scalar
from nitypes.waveform import AnalogWaveform, Timing

def simulate_voltage_regulator(input_voltage, time_ms, target_voltage=5.0):
    """
    Simulates the output voltage of a voltage regulator over time.
    """
    output = np.zeros_like(time_ms, dtype=float)
    startup_delay = 50  # ms
    ramp_duration = 200  # ms
    overshoot = 0.2 if input_voltage >= target_voltage else 0.0
    max_output = min(input_voltage, target_voltage + overshoot)

    for index, t in enumerate(time_ms):
        if t < startup_delay:
            output[index] = 0
        elif t < startup_delay + ramp_duration:
            ramp_progress = (t - startup_delay) / ramp_duration
            output[index] = ramp_progress * max_output
        else:
            # Simulate regulation behavior
            if input_voltage < target_voltage:
                output[index] = input_voltage
            elif input_voltage <= 8:
                output[index] = target_voltage + overshoot * np.exp(-(t - startup_delay - ramp_duration)/300)
            elif input_voltage <= 10:
                output[index] = target_voltage + 0.5 + 0.2 * np.sin(0.01 * t)
            else:
                output[index] = target_voltage + 1.0 + 0.5 * np.sin(0.01 * t)
    return output

# Time in milliseconds
time_ms = np.linspace(0, 1000, 500)
step = Step(step_name="Initial step", test_result_id=test_result_id)
step_id = client.create_step(step)

# Input voltages to simulate
input_voltages = [4.5, 7.0, 9.0, 12.0]

# Simulate each input voltage
for vin in input_voltages:
    waveform_data = simulate_voltage_regulator(vin, time_ms)
    waveform = AnalogWaveform(
            sample_count=len(waveform_data),
            raw_data=np.array(waveform_data),
            timing=Timing.create_with_regular_interval(
                ht.timedelta(seconds=1e-3),
                ht.datetime.now(timezone.utc)
            )
        )
    published_measurement = client.publish_measurement(
        measurement_name="voltage_response",
        value=waveform,
        step_id=step_id,
    )
    scalar = Scalar(value=vin, units="V")
    published_condition = client.publish_condition(
        "input_voltage",
        "integer",
        scalar,
        step_id
    )


## Read Published Data

Read back the published data and plot them using the parameter (condition) values and the timing data from the stored `AnalogWaveforms` and display them in a plotly graph.

In [ ]:
from nitypes.vector import Vector

# Create Plotly figure
fig = go.Figure()
colors = ['blue', 'green', 'orange', 'red']

conditions = client.query_conditions(odata_query=f"$filter=stepid eq {step_id}")
condition = next(iter(conditions), None)
condition_values = []
if condition is not None:
    condition_data = client.read_data(condition, expected_type=Vector)
    units = condition_data.units
    for val in condition_data:
        condition_values.append(str(val) + units)
measurements = client.query_measurements(odata_query=f"$filter=stepid eq {step_id}")
sorted_measurements = sorted(measurements, key=lambda m: m.parametric_index)
for measurement in sorted_measurements:
    waveform = client.read_data(measurement, expected_type=AnalogWaveform)
    color = colors[measurement.parametric_index]
    timing = waveform.timing
    fig.add_trace(go.Scatter(
        x=np.arange(len(waveform.raw_data)) * waveform.timing.sample_interval.total_seconds() * 1000,
        y=waveform.raw_data,
        mode='lines',
        name=f'Input {condition_values[measurement.parametric_index]}',
        line=dict(color=color)
    ))


# Update layout
fig.update_layout(
    title='Voltage Regulator Output Simulation',
    xaxis_title='Time (ms)',
    yaxis_title='Output Voltage (V)',
    legend_title='Input Voltage',
    template='plotly_white'
)

fig.show()